Full pandas dataframe created for all shep race results

todo:
create graph of box number vs place
create lin model of distance box vs time/place?

idea:
clustering analysis of dog to predict dogs that will behave the same  

working:
currently working with all split time correct in r
creating dog index in r to hold previous race results


In [17]:
import os
import xml.etree.ElementTree as ET
import pandas as pd
import tqdm
import pickle

In [16]:
#Loading race results database
os.chdir(r'C:\Users\Nick\Documents\GitHub\grvmodel\grv scraper\full race results')
file_list = os.listdir()
path = os.getcwd()


#print(file_list)


In [10]:
#master_list = pd.read_csv('masterfile.csv')

# not needed unless needs to be regenerated
master_list = result_getter(file_list, path)


100%|██████████| 4811/4811 [00:30<00:00, 157.64it/s]


In [4]:
def result_labels(file_list, path):
    label_list = []
    for i in file_list:
        root = ET.parse(i).getroot()
        
        for race in root.iter('Race'):
            race_id = race.attrib["id"]
            print("race id = ",race_id)
            
            places = []
            
            for dog in race.iter('Dog'):
                places.append(dog[0].text)
            print(places)
        break
            

In [9]:
def result_getter(file_list, path):
    master_list = []
    
    for i in tqdm.tqdm(file_list):
        root = ET.parse(i).getroot()

        track = root[0]
        #print("track =", track.text)
        date = root[1]
        #print("date =", date.text)


        for race in root.iter('Race'):
            #print("race id = ",race.attrib["id"])
            #print("Distance =", race[3].text, "\n")
            #print("Grade =", race[4].text, "\n")
            #print("place name box split split_margin runtime")

            places = []
            ids = []
            boxes = []
            splittimes = []
            split_margin = []
            split_position = []
            runtimes = []

            for dog in race.iter('Dog'):
                if not dog.attrib["id"]:
                    continue
                if not dog[0].text:
                    continue
                if (dog[0].text in "RSBFNPTBD")or('=' in dog[0].text):
                    #print("scratched check working")
                    continue
                else:
                    places.append(dog[0].text)
                    ids.append(dog.attrib["id"])
                    boxes.append(dog[3].text)
                    splittimes.append(dog[13].text)
                    runtimes.append(dog[14].text)

            for i in range(len(splittimes)):
                if splittimes[0] == None:
                    split_margin.append(pd.NaT)
                    continue
                try:
                    split_margin.append("%.2f" % (float(splittimes[i])-float(min(splittimes))))
                    
                                        #trying to get split position
                except:
                    split_margin.append(pd.NaT)
                    
                    
            for i in range(len(places)):
                try:
                    pass
                    #print(places[i], ids[i], boxes[i], splittimes[i], "%.2f" % split_margin[i], runtimes[i])
                except :
                    pass
                    #print(places[i], ids[i], boxes[i], splittimes[i], split_margin[i], runtimes[i])

            #print("\n------------------------\n")
            zipall = list(zip(places,ids,boxes,splittimes,split_margin,runtimes))
            #print(zipall)
            for i in range(len(zipall)):
                zipall[i] = (race.attrib["id"],track.text, race[3].text, race[4].text,) + zipall[i]
                master_list.append(zipall[i])

    return master_list

In [13]:
col_names = ["race_id", "track","dist","grade", "place", "dog_id", "box", "split_times", "split_margins", "run_time"]
df = pd.DataFrame(master_list, columns = col_names)
df.place = df.place.astype(float)
df.place.unique()

array([1., 2., 3., 4., 5., 6., 7., 8.])

In [15]:
print(df.size)
df = df.dropna()
print(df.size)
df.to_csv("masterfile.csv")

with open("results-df.npy", "wb") as fp:   #Pickling
    
    pickle.dump(df, fp)

4178470
4151160


NameError: name 'pickle' is not defined

In [14]:
df.grade.value_counts()

Grade 5                            92157
Mixed 4/5                          38487
Maiden                             32138
Grade 5 T3                         26966
Mixed 6/7                          22233
Grade 7                            17453
Restricted Win                     16629
Grade 5 Heat                       14403
Tier 3 - Maiden                    13181
Maiden Heat                        12606
Restricted Win Heat                11648
Tier 3 - Grade 7                   10824
Grade 6                            10476
S/E Heat                            8932
Free For All                        8421
Mixed 6/7 Heat                      8317
Tier 3 - Restricted Win             6760
Tier 3 - Grade 6                    6703
Grade 5 No Penalty                  6357
Aged Prize Money                    5449
Grade 5 Final                       4533
Special Event                       4440
Mixed 3/4                           3869
Restricted Win Final                3782
Maiden Final    

In [13]:
#generates adjusted place to transfer to form results

dfsorted = df.sort_values(['box'])
dfsorted['place_adj'] = 1/(dfsorted["place"])
grouped = dfsorted.groupby(['race_id'])

for group in grouped:
    
    print(group[1])
    break
 

         race_id      track  dist         grade  place      dog_id  box  \
312035  12539914  Traralgon  525m  Maiden Final    8.0   109032142    1   
312031  12539914  Traralgon  525m  Maiden Final    4.0   145856635    2   
312033  12539914  Traralgon  525m  Maiden Final    6.0  2131520028    3   
312030  12539914  Traralgon  525m  Maiden Final    3.0   167542442    4   
312029  12539914  Traralgon  525m  Maiden Final    2.0   128006011    5   
312034  12539914  Traralgon  525m  Maiden Final    7.0  1199430051    6   
312032  12539914  Traralgon  525m  Maiden Final    5.0   159734832    7   
312028  12539914  Traralgon  525m  Maiden Final    1.0   115367782    8   

        split_times  split_margins  run_time  place_adj  
312035         5.06           0.20     32.31   0.125000  
312031         5.06           0.20     31.53   0.250000  
312033         5.06           0.20     31.70   0.166667  
312030         4.90           0.04     31.31   0.333333  
312029         5.04           0.18

In [15]:
dfsorted.to_csv("masterfilenew.csv")
